In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
#main url of the site
url = 'https://www.socialmediatoday.com/?page='

In [3]:
urls = []
# scraping the first five pages of the website for article links
for i in range(1, 6):
    urls.append(url + str(i))

In [4]:
urls

['https://www.socialmediatoday.com/?page=1',
 'https://www.socialmediatoday.com/?page=2',
 'https://www.socialmediatoday.com/?page=3',
 'https://www.socialmediatoday.com/?page=4',
 'https://www.socialmediatoday.com/?page=5']

In [5]:
def get_article_links(url, links):
    raw_content = requests.get(url)
    soup = BeautifulSoup(raw_content.text, 'html.parser')
#     links = []
    
    for link in soup.find_all('a', attrs = {'href': re.compile('^/news/'), 'class': re.compile('^analytics t-dash-feed-item-\d')}):
#         print(link.text)
        links.append(link['href'])
        
#     return links

In [7]:
links = []
for i in range(len(urls)):
    get_article_links(urls[i], links)

In [8]:
links

['/news/meta-releases-new-insights-into-its-evolving-efforts-to-detect-coordinated/617531/',
 '/news/reddits-experimenting-with-nft-profile-pictures-leaning-into-audience-tre/617542/',
 '/news/new-survey-shows-that-social-media-users-are-increasingly-concerned-about-d/617548/',
 '/news/12-helpful-seo-tools-for-your-brand-in-2022-infographic/617547/',
 '/news/twitter-launches-new-nft-profile-image-display-option-to-twitter-blue-subsc/617457/',
 '/news/instagram-expands-video-remix-option-to-all-videos-not-just-reels-clips/617465/',
 '/news/meta-plans-to-establish-an-nft-marketplace-expanding-beyond-profile-pictur/617453/',
 '/news/instagram-will-now-reduce-the-reach-of-posts-that-are-likely-to-contain-b/617478/',
 '/news/snapchat-shares-new-insights-into-user-engagement-around-valentines-day-i/617474/',
 '/news/messenger-kids-launches-new-digital-literacy-initiative-to-better-educate-y/617470/',
 '/news/meta-announces-planned-integration-of-whatsapp-into-its-workplace-platform/617480/',

In [9]:
#since the urls we got are not complete, we are appending the initial part of the site
append_str = 'https://www.socialmediatoday.com'

for i in range(len(links)):
    links[i] = append_str + links[i]
    
print(links[0])

https://www.socialmediatoday.com/news/meta-releases-new-insights-into-its-evolving-efforts-to-detect-coordinated/617531/


In [10]:
len(links)

125

In [11]:
#removing duplicate links
links = list(set(links))
len(links)

75

In [12]:
def get_article(soup, article):
    #heading 
    heading = soup.find('h1', {'class': 'display-heading-04'})
    
    #published date
    published = soup.find('span', {'class': 'published-info'})
    
    #author
    author = soup.find('a', {'rel': 'author'})
    
    #content
    content = soup.find('div', {'class': 'large medium article-body'})
    
    text_content = content.text.split('.')
    for i in range(len(text_content)):
        text_content[i] = text_content[i].replace('\n', '')
    final_content = '. '.join(text_content)
    
    #tags
    tags = soup.find('a', {'class': 'topic analytics t-article-end-topic'})
    
    article['heading'].append(heading.text.strip('\n'))
    article['published'].append(published.text.strip('\n'))
    article['author'].append(author.text)
    article['content'].append(final_content)
    article['tags'].append(tags.text)

In [13]:
article = {'heading': [], 'author': [], 'published': [], 'content': [], 'tags': []}

for i in range(0, len(links)):
    req = requests.get(links[i])
    if req.status_code == 200:
        soup = BeautifulSoup(req.text, 'html.parser')
        get_article(soup, article)
        
# print(article['heading'])

In [14]:
len(article['heading']) ##at the time of error

75

In [15]:
import pandas as pd

df = pd.DataFrame(article)
df.head()

,heading,author,published,content,tags
0,TikTok Shares New Insights into the Importance...,Andrew Hutchinson,"Published Jan. 4, 2022","Unlike other video platforms, which have tradi...","Content Marketing,"
1,Instagram Will Now Reduce the Reach of Posts T...,Andrew Hutchinson,"Published Jan. 20, 2022",Instagram is implementing new measures that wi...,Social Media Updates
2,TikTok Shares New Data on the Value of Collabo...,Andrew Hutchinson,"Published Jan. 10, 2022",The key to winning on TikTok is aligning with ...,"Digital Strategy,"
3,Pinterest Adds 'Profiles' Tab to Search Result...,Andrew Hutchinson,"Published Jan. 7, 2022",Pinterest is looking to improve creator discov...,"Digital Strategy,"
4,Twitter Testing New Search Prompts Above the M...,Andrew Hutchinson,"Published Jan. 10, 2022","Here’s a relatively small, but potentially val...",Social Media Updates


In [16]:
df.to_csv('articles.csv')